# Initialization

## Initializing a virtual set-up

In [ ]:
from pycqedscripts.init.demo.virtual_ATC75_M136_S17HW02_PQSC import *

## Setting the data directories

In [ ]:
MC.datadir(r'data')  # data dir for virtual measurements
a_tools.datadir = MC.datadir()

## Importing modules
Importing relevant modules (including the one containing new `AutomaticCalibrationRoutine` class)

In [ ]:
from pycqed.measurement.calibration import automatic_calibration_routines as acr

## Initializing the device for automated routines

Most of the parameters defining routines are stored in configuration files. They have to be loaded at first.

In [ ]:
# FIXME this section should be moved in regular init at some point
dev.MWGs = MWGs
dev.TWPAs = TWPAs
# If settings_default_folder is None, it uses the default PycQED settings 
# in pycqed.measurement.calibration.autocalib_default_settings
settings_default_folder = None 
settings_setup_folder = r'..\..\..\pycqed_scripts\pycqedscripts\init\demo\autocalib_settings\Setup'
settings_sample_folder = r'..\..\..\pycqed_scripts\pycqedscripts\init\demo\autocalib_settings\ATC124_M154_S17_4'
settings = acr.SettingsDictionary()
settings.load_settings_from_file(
    settings_default_folder=settings_default_folder,
    settings_setup_folder=settings_setup_folder,
    settings_sample_folder=settings_sample_folder)
dev.autocalib_settings = settings

## Setting qubits

In [ ]:
qb = qb1
qubits = [qb]

# Routines

##  Single qubit calibration

Single qubit calibration is a routine to recalibrate qubits and reset the setup to a default state.

First, just call the routine with the parameters set in the configuration files set above and all qubits. There might be some analysis errors since the data on the virtual setup are random.

In [ ]:
sqc = SingleQubitCalib(dev=dev)

Now let's set some configuration parameters for the routine via keyword arguments.

In [ ]:
sqc = acr.SingleQubitCalib(dev=dev, qubits=[qb1],  transition_names=['ge', 'ef'])

Lastly, let's define a user dictionary similar to the configuration files as a last option to set user parameters.

In [ ]:
# Turn on some steps with a user dictionary
settings_user = {
    'SingleQubitCalib': {
        'General': {
           't1': True,
           'echo_small_AD': True,
           'in_phase_calib': True
        },

    }
}

In [ ]:
sqc = acr.SingleQubitCalib(dev=dev, qubits=[qb1], settings_user=settings_user)

## Pi-pulse calibration

The PiPulseCalibration routine is a sequency of one Rabi and one Ramsey measurement. This can be useful to determine the transition frequency of a particular transition.

In [ ]:
pipulse_calib = acr.PiPulseCalibration(dev = dev,
                                   qubits = qubits, 
                                   autorun=False,
                                   update=True,
                                  )

In [ ]:
pipulse_calib.run()

## Finding frequency

The FindFrequency routine is a routine that iterates between Rabi and Ramsey experiments to determine the transition frequency. It is an extension of the Pi-pulse calibration, and makes use of an additional Decision step to see if the Ramsey outcomes have "converged". That is, if the current Ramsey experiment lead to the same result as a previous Ramsey experiment (up to a given distance `allowed_delta_f`).

In [ ]:
ff = acr.FindFrequency(qubits=[qb1], dev=dev, autorun=False, update=True)

In [ ]:
ff.run()

##  Hamiltonian fitting

This routine performs the Hamiltonian fitting procedure. 

**NOTE** the implementation of the Hamiltonian fitting routine makes use of flux-pulse assisted read-out. This technique requires the parameter `qb.flux_amplitude_bias_ratio()` to be set. For this demonstration, we will set it to some arbitrary value but beware that on a real setup the true value is required (depends on the electronics to the quantum chip).

In [ ]:
qb.flux_amplitude_bias_ratio(42)

### with prior model

#### Defining the routine

In [ ]:
pre_routine_model = {'Ej_max': 18925634578.803883, 
                     'E_c': 179311053.4831606, 
                     'asymmetry': 0.3574218197407885, 
                     'coupling': 165221330.37944722, 
                     'fr': 7350129526.414872, 
                     'V_per_phi0': 5.1435508397790715, 
                     'dac_sweet_spot': -1.499}

qb.fit_ge_freq_from_dc_offset(pre_routine_model)

In [ ]:
hamfit = acr.HamiltonianFitting(
    qubit = qb, 
    dev=dev,
    fluxlines_dict = fluxlines_dict,
    use_prior_model = True,
    autorun=False)

#### Executing the routine

In [ ]:
hamfit.run()

#### Resulting Hamiltonian model

In [ ]:
qb.fit_ge_freq_from_dc_offset()

Comparing with the old model:

In [ ]:
pre_routine_model

### without prior model

#### Defining the routine

Starting with an empty model. The only necessary entries are 'V_per_phi0' and 'dac_sweet_spot' for flux-pulse-assisted readout.

In [ ]:
qb.fit_ge_freq_from_dc_offset({
    'V_per_phi0': 5.1435508397790715, 
    'dac_sweet_spot': -1.499
}) 

In [ ]:
measurements = {0: ('ge', 'ef'), -0.5: ('ge',), -0.25: ('ge',), -0.3:('ge',)}

In [ ]:
flux_to_voltage_and_freq_guess={0:(-0.5, 6e9), -0.5:(-3.5, 4.5e9),-0.3:(-3, 4.75e9)}

In [ ]:
hamfit = acr.HamiltonianFitting(
    qubit = qb, 
    dev=dev, 
    fluxlines_dict = fluxlines_dict, 
    use_prior_model = False,
    measurements = measurements,
    flux_to_voltage_and_freq_guess=flux_to_voltage_and_freq_guess, 
    autorun=False, 
)

In [ ]:
hamfit.ss1_flux

In [ ]:
hamfit.ss2_flux

In [ ]:
hamfit.other_fluxes_with_guess

#### Executing the routine

In [ ]:
hamfit.run()

#### Resulting Hamiltonian model

In [ ]:
qb.fit_ge_freq_from_dc_offset()